<img src="../../../img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">Introduction to the Web Time Series Service (WTSS) in R</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/R/wtss/wtss-introduction.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br>

<div style="text-align: center;font-size: 90%;">
    Felipe Carvalho de Souza<sup><a href="https://orcid.org/0000-0002-5826-1700"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: March 24, 2021
</div>

<br>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives an overview on how to use the WTSS service to extract time series from <em>Earth Observation Data Cubes</em>.
</div>    

<br>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Vinhas, L.; Queiroz, G. R.; Ferreira, K. R.; Camara, G. <a href="http://www.seer.ufu.br/index.php/revistabrasileiracartografia/article/view/44004" target="_blank">Web Services for Big Earth Observation Data</a>. Revista Brasileira de Cartografia, v. 69, n. 5, 18 maio 2017.
    </div>
</div>

# Introduction
<hr style="border:1px solid #0077b9;">

The **W**eb **T**ime **S**eries **S**ervice (WTSS) is a lightweight web service for handling time series data from remote sensing imagery. Given a location and a time interval you can retrieve the according time series as a list of real values.


In WTSS a coverage is a three dimensional array associated to spatial and temporal reference systems (Figure 1).

<div align="center">
    <img src="../../../img/image-time-series.png" width="240" />
    <br/>
    <b>Figure 1</b> - A coverage as a three dimensional array.
</div>

WTSS is based on three operations:

- ``list_coverages``: returns the list of all available coverages in the service.

- ``describe_coverage``: returns the metadata of a given coverage.

- ``time_series``: query the database for the list of values for a given location and time interval.

This Jupyter Notebook shows how to use WTSS in Python with Brazil Data Cube data.

# R Client API
<hr style="border:1px solid #0077b9;">

The command below installs the additional packages that are required to run this Jupyter Notebook.

In [ ]:
# Remove '#' to install the packages
# install.packages(c("remotes", "magrittr", "dplyr", "tibble", "ggplot2"))

For running the examples in this Jupyter Notebook you will need to install the [Rwtss client for R](https://github.com/e-sensing/Rwtss). The command below installs `Rwtss` direct from the code repository of the Brazil Data Cube project using [remotes](https://CRAN.R-project.org/package=remotes).

In [ ]:
# Remove '#' to install the package
# remotes::install_github("e-sensing/Rwtss", upgrade = "never")

In [ ]:
library(Rwtss)    # Package wtss
library(magrittr) # Package to use pipe operator %>%
library(dplyr)    # Package to handle with data frame
library(tibble)   # Package to represent data frame structure
library(ggplot2)  # Package to create graphics 

Before we start, let's create a variable that corresponds to the URL of the INPE WTSS service.

In [ ]:
wtss_inpe <- "https://brazildatacube.dpi.inpe.br/esensing/wtss"

# Listing the Available Data Products
<hr style="border:1px solid #0077b9;">

The function `list_coverages` allows to list the available coverages:

In [ ]:
Rwtss::list_coverages(wtss_inpe)

The names returned by property can be used in subsequent operations.

The complete documentation of `list_coverages` function can be found using `?Rwtss::list_coverages`. Like the cell presented bellow:

In [ ]:
# ?Rwtss::list_coverages

# Retrieving the Metadata of a Data Product
<hr style="border:1px solid #0077b9;">

It is possible to access the metadata of a specific coverage with the `describe_coverage` function:

**MOD13Q1 product**

In [ ]:
Rwtss::describe_coverage(wtss_inpe, name = "MOD13Q1")

**MOD13Q1_M product**

In [ ]:
coverage <- Rwtss::describe_coverage(wtss_inpe, name = "MOD13Q1_M")

The coverage metadata includes its range in the spatial and temporal dimensions, the list of attributes associated to the cells, and the timeline.


Let's inspect the metadata with more detail:

In [ ]:
names(coverage)

In [ ]:
coverage$name

In [ ]:
coverage$satellite

In [ ]:
coverage$sensor

In [ ]:
cat("nrows: ", coverage$nrows)
cat("\n")
cat("ncols: ", coverage$ncols)

In [ ]:
cat("xmin: ", paste(coverage$xmin))
cat("\n")
cat("xmax: ", paste(coverage$xmax))
cat("\n")
cat("ymin: ", paste(coverage$ymin))
cat("\n")
cat("ymax: ", paste(coverage$ymax))

In [ ]:
start_date <- min(coverage$timeline[[1]])
end_date <- max(coverage$timeline[[1]])

sprintf("Interval range: [%s, %s]", start_date, end_date)

For more, you can find the complete documentation of `describe_coverage` function in `?Rwtss::describe_coverage`

# Retrieving the Time Series
<hr style="border:1px solid #0077b9;">

In order to retrieve the time series for attributes `red` and `nir`, in the location of `latitude -12` and `longitude -54` from `January 1st, 2001` to `December 31st, 2001`, use the `time_series` function:

In [ ]:
ts <- Rwtss::time_series(wtss_inpe,
                         attributes = c("red", "nir"), 
                         latitude   = -12.0, 
                         longitude  = -54.0, 
                         name       = "MOD13Q1",
                         start_date = "2001-01-01",
                         end_date   = "2001-12-31")

The attributes returned from function `time_series` are listed in the cell below:

In [ ]:
tibble::glimpse(ts)

Each time series can be accessed by the attribute `time_series`, e.g.:`ts$time_series[[1]]$nir`

In [ ]:
head(ts$time_series[[1]])

For more, you can find the complete documentation of `time_series` function in `?Rwtss::describe_coverage`

# Visualizing the Time Series
<hr style="border:1px solid #0077b9;">

The returned time series can be visualized using the `plot` function. Let's visualize the time series we have seen before:

In [ ]:
plot(ts)

# References
<hr style="border:1px solid #0077b9;">

- [WTSS Server](https://github.com/brazil-data-cube/wtss)

- [WTSS OpenAPI 3 Specification](https://github.com/brazil-data-cube/wtss-spec)

- [R Client Library for Web Time Series Service - Github Repository](https://github.com/e-sensing/wtss)